# 手眼标定

同济子豪兄 2024-5-13

## 手眼标定的原理

已知图像上两个点的【像素坐标】和【机械臂坐标】，就可以通过【线性插值】，建立像素坐标到机械臂坐标的映射关系。输入图像任意一点的像素坐标，eye2hand函数就能转换为机械臂坐标。让机械臂移动到图像上的同一个位置。

在utils_robot.py里修改eye2hand函数，填好八个数字坐标。

## 导入工具包

In [1]:
from pymycobot import MyCobot280Socket
import time

import cv2
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## 连接机械臂

In [2]:
mc = MyCobot280Socket("192.168.31.175", 9000)

## 设置运动模式为插补

In [3]:
mc.set_fresh_mode(0)
print(mc.get_fresh_mode())

0


## 机械臂归零

In [29]:
mc.send_angles([0, 0, 0, 0, 0, 0], 40)

-1

## 第一步：移动至俯视姿态

俯视姿态一（关节）：[0, 0, -88, 0, 0, 45]

俯视姿态二（坐标）：[13, -160, 212, 180, 3.31, -135.81]

俯视姿态二（关节）：[-62.13, 8.96, -87.71, -14.41, 2.54, -16.34]

In [18]:
# mc.send_coords([13, -160, 212, 180, 3.31, -135.81], 10)

mc.send_angles([-90, 0, -90, 0, 0, 45], 30)

-1

In [17]:
print(mc.get_angles())

[-62.05, 8.43, -90.35, -12.56, 2.81, -15.55]


In [24]:
# mc.send_angles([-62.13, 8.96, -87.71, -10.41, 2.54, -16.34], 10)

# mc.send_coord(1, 25, 20)

mc.send_angles([-79.1, -3.33, -84.28, 0, 1.49, -33.48], 20)

## 第二步：在白纸靠左下角的位置，画一个标定点

## 第三步：运行check_camera.py，打开摄像头实时画面

## 第四步：把白纸右上角对准画面右上角

## 第五步：白纸上边与底座、图像上边平齐，白纸下边与图像下边平齐

## 第六步：用夹子固定白纸，分别夹左上角和右下角。（把麦克风线也固定）

## 第七步：通过鼠标点选，获取白纸左下角标定点，在图像上的像素坐标

In [35]:
# 第一个标定点的像素坐标
cali_1_im = [63, 409]

## 第八步：控制机械臂，移动至左下角第一个标定点

In [44]:
# 移动到标定点附近
mc.send_coords([130, -70, 94.7, 178.92, 4.56, -135.57], 10)
time.sleep(1)

In [11]:
mc.set_free_mode(1)
print(mc.get_free_mode())

AttributeError: 'MyCobot280Socket' object has no attribute 'get_free_mode'

In [13]:
mc.power_on()
mc.release_all_servos()



-1

In [79]:
mc.power_on()


[-73.6, -202.4, 89.8, 177.72, 5.64, -135.65]

In [72]:
X = -73.6

In [78]:
X -= 5
mc.send_coord(1, X, 20)
X

-75

In [86]:
Y = -151

In [89]:
Y += 10
mc.send_coord(2, Y, 20)
Y

-121

In [92]:
Z = 90

In [100]:
Z -= 3
mc.send_coord(3, Z, 20)

-1

In [161]:
mc.get_coords()

[127.1, -77.8, 92.0, 178.15, 3.91, -135.81]

- 记下机械臂坐标
[127.1, -77.8, 92.0, 178.15, 3.91, -135.81]

In [108]:
# 第一个标定点的机械臂坐标
cali_1_mc = 127.1, -77.8

## 第九步：控制机械臂，移动至右上角第二个标定点

In [46]:
# 第二个标定点的像素坐标
cali_2_im = [493, 39]

In [113]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(1)

In [51]:
# 移动到第二个标定点附近
mc.send_coords([-30, -255, 94.7, 178.92, 4.56, -135.57], 10)
time.sleep(1)

In [52]:
mc.get_coords()

[-32.1, -252.2, 84.2, 176.97, 6.3, -136.04]

In [105]:
Z = 90

In [106]:
Z -= 0
mc.send_coord(3, Z, 20)
time.sleep(1)

In [115]:
mc.get_coords()

[104.6, -250.8, 85.2, 178.71, 3.29, -136.05]

In [116]:
X = 110

In [120]:
X -= 0
mc.send_coord(1, X, 20)
time.sleep(1)

In [122]:
Y = -249.4

In [123]:
Y += 5 
mc.send_coord(2, Y, 20)
time.sleep(1)

In [171]:
mc.get_coords()

[-63.4, -252.8, 87.9, 177.6, 5.79, -135.24]

In [72]:
# 第二个标定点的机械臂坐标
cali_2_mc = [-63.4, -252.8]

In [172]:
[-63.4, -252.8, 87.9, 177.6, 5.79, -135.24]

[-63.4, -252.8, 87.9, 177.6, 5.79, -135.24]

In [ ]:
cali_1_im = [45, 379]
cali_1_mc = []
cali_2_im = [524, 10]
cali_2_mc = []

In [23]:
print(mc.get_coords())

[15.3, -162.5, 206.5, 175.02, 3.86, -137.75]


## 第十步：通过插值，获取图像任意像素坐标对应的机械臂坐标

In [30]:
# 整理两个标定点的坐标
cali_1_im = [64, 434]                       # 左下角，第一个标定点的像素坐标，要手动填！
cali_1_mc = [122.4, -98.0]                  # 左下角，第一个标定点的机械臂坐标，要手动填！
cali_2_im = [442, 20]                         # 右上角，第二个标定点的像素坐标
cali_2_mc = [-33.7, -271.3]                   # 右上角，第二个标定点的机械臂坐标，要手动填！

In [61]:
mc.send_angles([-79.1, -3.33, -84.28, 0, 1.49, -33.48], 20)

-1

In [7]:
mc.power_off()

-1

In [8]:
mc.power_on()

-1

In [9]:
print(mc.get_coords())

[-33.7, -271.3, 85.7, 179.52, -1.62, -135.88]


In [175]:
p# 指定点在图像中的像素坐标
X_im = 409
Y_im = 248

In [176]:
X_cali_im = [cali_1_im[0], cali_2_im[0]]     # 像素坐标
X_cali_mc = [cali_1_mc[0], cali_2_mc[0]]     # 机械臂坐标

X_mc = int(np.interp(X_im, X_cali_im, X_cali_mc))

Y_cali_im = [cali_2_im[1], cali_1_im[1]]     # 像素坐标，先小后大
Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]     # 机械臂坐标，先大后小

Y_mc = int(np.interp(Y_im, Y_cali_im, Y_cali_mc))

In [177]:
X_mc

33

In [178]:
Y_mc

-159

## 让机械臂移动至该点吸取

In [179]:
mc.send_coords([X_mc, Y_mc, 200, -178.24, 1.68, -134.33], 20)
time.sleep(3)

In [143]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

## 封装手眼标定函数

In [180]:
import numpy as np
def eye2hand(X_im=160, Y_im=120):
    '''
    输入目标点在图像中的像素坐标，转换为机械臂坐标
    '''

    # 整理两个标定点的坐标
    cali_1_im = [63, 409]                       # 左下角，第一个标定点的像素坐标，要手动填！
    cali_1_mc = [127.1, -77.8]                  # 左下角，第一个标定点的机械臂坐标，要手动填！
    cali_2_im = [493, 39]                         # 右上角，第二个标定点的像素坐标
    cali_2_mc = [-63.4, -252.8]                   # 右上角，第二个标定点的机械臂坐标，要手动填！
    
    X_cali_im = [cali_1_im[0], cali_2_im[0]]     # 像素坐标
    X_cali_mc = [cali_1_mc[0], cali_2_mc[0]]     # 机械臂坐标
    Y_cali_im = [cali_2_im[1], cali_1_im[1]]     # 像素坐标，先小后大
    Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]     # 机械臂坐标，先大后小

    # X差值
    X_mc = int(np.interp(X_im, X_cali_im, X_cali_mc))

    # Y差值
    Y_mc = int(np.interp(Y_im, Y_cali_im, Y_cali_mc))

    return X_mc, Y_mc

In [182]:
eye2hand(X_im=304, Y_im=77)

In [81]:
eye2hand(X_im=320, Y_im=240)

(66, -173)

## 第十一步：验证标定效果

In [82]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

In [184]:
# 移动至俯视姿态
mc.send_angles([-62.13, 8.96, -87.71, -10.41, 2.54, -16.34], 20)
time.sleep(3)

In [36]:
# 运行`camera_check.py`，用鼠标点选图像中的某个点，获取像素坐标
X_im, Y_im = 267, 97

In [37]:
# 手眼标定转换为机械臂坐标
X_mc, Y_mc = eye2hand(X_im, Y_im)

In [183]:
# 控制机械臂移动到这个点，看是否准确
mc.send_coords([X_mc, Y_mc, 100, -178.24, 1.68, -134.33], 20)
time.sleep(3)